In [3]:
!pip install -U pandasql

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26768 sha256=9a1977244685d960419473394c34a77384bb311dbc77398c8820cce721453160
  Stored in directory: c:\users\eberl\appdata\local\pip\cache\wheels\63\e8\ec\75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07
Successfully built pandasql


In [4]:
from pandasql import sqldf
import pandas as pd
from sklearn import datasets

In [21]:
Orders= 'C:/Users/eberl/Documents/Dat512/Ecommerce Project/Order_items.csv'
Events= 'C:/Users/eberl/Documents/Dat512/Ecommerce Project/Events.csv'
Users= 'C:/Users/eberl/Documents/Dat512/Ecommerce Project/Users.csv'
OrdersDF=pd.read_csv(Orders)
EventsDF=pd.read_csv(Events)
UsersDF=pd.read_csv(Users)

In [22]:
q = "SELECT * FROM OrdersDF"
sqldf(q, globals())

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,75713,52275,41829,13606,204495,Shipped,2023-03-26 02:34:05.000000 UTC,2023-03-29 02:22:00.000000 UTC,None,None,2.5
1,103479,71377,57213,13606,279368,Shipped,2023-04-04 13:58:37.000000 UTC,2023-04-02 06:38:00.000000 UTC,None,None,2.5
2,63235,43638,34774,13606,170721,Complete,2020-11-20 14:37:22.000000 UTC,2020-11-23 17:08:00.000000 UTC,2020-11-27 19:52:00.000000 UTC,None,2.5
3,105541,72790,58330,13606,284943,Returned,2020-06-27 12:58:45.000000 UTC,2020-06-28 14:46:00.000000 UTC,2020-07-02 11:26:00.000000 UTC,2020-07-03 23:27:00.000000 UTC,2.5
4,44203,30542,24381,13606,119442,Cancelled,2021-03-01 09:16:55.000000 UTC,None,None,None,2.5
...,...,...,...,...,...,...,...,...,...,...,...
67574,99702,68723,55078,21521,269295,Complete,2022-06-30 07:08:27.000000 UTC,2022-07-03 03:00:00.000000 UTC,2022-07-08 00:01:00.000000 UTC,None,118.0
67575,100794,69505,55714,13035,272200,Complete,2022-05-01 13:41:25.000000 UTC,2022-05-03 09:49:00.000000 UTC,2022-05-03 12:31:00.000000 UTC,None,118.0
67576,106020,73112,58602,9915,286218,Complete,2023-01-29 02:34:41.000000 UTC,2023-01-31 09:29:00.000000 UTC,2023-02-03 20:34:00.000000 UTC,None,118.0
67577,109489,75472,60512,4860,295629,Complete,2023-04-12 09:54:32.000000 UTC,2023-04-10 13:33:00.000000 UTC,2023-04-13 17:56:00.000000 UTC,None,118.0


In [23]:
q = "SELECT count(DISTINCT(product_id)) as total FROM OrdersDF"
sqldf(q, globals())

,total
0,10871


There are ~11k unique products in this data set
we'll want to look into cutting this list down by the worst selling 
20% (~2200 unique products)

In [25]:
q = "SELECT product_id, Count(product_id) as total_sold, sale_price FROM OrdersDF GROUP BY product_id,sale_price ORDER BY total_sold ASC LIMIT 2200"
sqldf(q, globals())

,product_id,total_sold,sale_price
0,112,1,17.5
1,116,1,24.5
2,132,1,30.0
3,852,1,25.0
4,897,1,92.5
...,...,...,...
2195,12101,4,42.0
2196,12106,4,40.0
2197,12118,4,21.0
2198,12133,4,36.0


Cutting down the list of products,
Here we have the 2200 worst selling products by total item sold, along with price
If we had more info on what these specific items were, we could dive into if these 
are possibly priced too high based on the item in question

Now I'll do a similar analysis of products by their total web views

In [26]:
q = "SELECT * FROM EventsDF"
sqldf(q, globals())

,id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type
0,2060642,NaN,2,59c58d57-6a0b-45c3-9d31-be38cfbfb447,2021-03-30 01:10:00.000000 UTC,93.89.195.100,Bogatynia,Dolnośląskie,59,Firefox,Email,/cart,cart
1,1864178,NaN,3,a43078a4-1162-402b-8b5a-63c1ca9a6698,2020-01-25 05:29:00.000000 UTC,130.225.144.226,Bogatynia,Dolnośląskie,59,Chrome,Adwords,/cart,cart
2,563756,43423.0,3,d4f9811c-685c-4c76-bb7b-832988a74634,2023-04-13 17:07:24.125348 UTC,92.165.182.20,Bogatynia,Dolnośląskie,59,IE,Email,/cart,cart
3,527503,40683.0,4,707d9b5c-522a-49af-ba0e-b0c668ccd5ef,2023-03-26 03:27:24.000000 UTC,106.12.125.113,Bogatynia,Dolnośląskie,59,Safari,Email,/cart,cart
4,2261172,NaN,2,2df3b32c-4d95-4a8a-a448-af16eb9fc9a6,2021-08-05 05:45:00.000000 UTC,39.160.249.193,Bogatynia,Dolnośląskie,59,Other,Email,/cart,cart
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51379,1284977,98873.0,11,34aa7ac4-a167-41de-a39e-84fb6fcccf8f,2023-03-09 06:14:05.000000 UTC,82.38.58.189,Maoming,Hainan,571100,Chrome,Email,/product/12581,product
51380,1677797,NaN,1,669dd284-397b-42e1-ab59-87c3df3605ab,2020-08-05 15:16:00.000000 UTC,155.129.20.204,Dongguan,Hainan,571101,Firefox,YouTube,/product/26152,product
51381,878690,67514.0,3,45b254c0-b897-4f9d-8dcb-767a164e7677,2023-03-10 00:07:02.000000 UTC,113.237.175.160,Dongguan,Hainan,571101,IE,Email,/product/18844,product
51382,390927,29916.0,5,2fffbd89-57e4-4c71-b786-65740fa46716,2023-02-12 14:35:36.000000 UTC,5.164.158.67,Dongguan,Hainan,571101,Firefox,YouTube,/product/25183,product


In [27]:
q = "SELECT distinct(browser)FROM EventsDF"
sqldf(q, globals())

,browser
0,Firefox
1,Chrome
2,IE
3,Safari
4,Other


In [28]:
q = "SELECT distinct(event_type)FROM EventsDF"
sqldf(q, globals())

,event_type
0,cart
1,home
2,cancel
3,product


I'm going to just focus on the event types cart and product, as the 
others dont show only web views, others include cancel purchase,
department or home page

In [31]:
q = "SELECT O.product_id, COUNT(O.product_id) total FROM EventsDF AS E LEFT JOIN OrdersDF AS O ON O.id = E.id WHERE E.event_type in ('cart', 'product') AND product_id is not null GROUP BY O.product_id ORDER BY total ASC LIMIT 5800;"
sqldf(q, globals())

,product_id,total
0,34,1
1,45,1
2,95,1
3,265,1
4,314,1
...,...,...
1040,27877,2
1041,27929,2
1042,27960,2
1043,28171,2


From the product sales and web views queries, the products that overlap in both views would be my recommendation 
for which products to remove from the companies circulation.

These products are those that sell the least amount while also being viewed online by users the least amount

Part 2

In [32]:
q = "SELECT * FROM UsersDF"
sqldf(q, globals())

,id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at
0,9884,Colleen,Ward,colleenward@example.org,48,F,Mie,249 Joseph Circle,513-0836,Suzuka City,Japan,34.851814,136.508713,Search,2022-08-23 11:40:00.000000 UTC
1,12266,Kathleen,Cruz,kathleencruz@example.net,26,F,Mie,388 Eric Forest Apt. 148,513-0836,Suzuka City,Japan,34.851814,136.508713,Search,2021-02-13 12:16:00.000000 UTC
2,1633,Leroy,Schneider,leroyschneider@example.org,31,M,Acre,9044 Andrade Road,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2020-11-24 17:13:00.000000 UTC
3,26151,Earl,Miller,earlmiller@example.net,30,M,Acre,62069 Rubio Garden,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2019-06-30 14:52:00.000000 UTC
4,31507,Gerald,Wyatt,geraldwyatt@example.org,69,M,Acre,9331 Schwartz Bypass Suite 042,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2020-02-07 00:42:00.000000 UTC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46139,90541,Steven,Sanchez,stevensanchez@example.net,35,M,Liaoning,2679 Williams Fords,115213,Zhenjiang,China,40.424994,122.290667,Organic,2019-04-19 10:46:00.000000 UTC
46140,92747,Natalie,Allen,natalieallen@example.org,14,F,Liaoning,5071 Colon Throughway,115213,Zhenjiang,China,40.424994,122.290667,Organic,2020-08-08 13:15:00.000000 UTC
46141,95501,Anita,Taylor,anitataylor@example.net,39,F,Liaoning,545 David Pine,115213,Zhenjiang,China,40.424994,122.290667,Search,2022-07-27 10:30:00.000000 UTC
46142,418,Brandi,Oneill,brandioneill@example.com,21,F,Liaoning,451 Dyer River Apt. 598,116023,Zhangzhou,China,38.884174,121.578857,Search,2021-10-31 03:23:00.000000 UTC


In [33]:
q = "SELECT distinct(gender), count(*) as gender_total FROM UsersDF GROUP BY gender"
sqldf(q, globals())

,gender,gender_total
0,F,23053
1,M,23091


Gender split is pretty much 50/50 in this data set of users

In [34]:
q = "SELECT distinct(country), count(*) as country_total FROM UsersDF  GROUP BY country ORDER BY country_total DESC"
sqldf(q, globals())

,country,country_total
0,China,18605
1,United States,10198
2,Brasil,5743
3,United Kingdom,4250
4,Japan,2406
5,South Korea,2123
6,Germany,1302
7,Belgium,860
8,Spain,410
9,France,231


This companie's data is mostly under China the US and Brazil,
with some other countries spread out in the remaining data

In [35]:
q = "SELECT * FROM EventsDF LIMIT 1000"
sqldf(q, globals())

,id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type
0,2060642,NaN,2,59c58d57-6a0b-45c3-9d31-be38cfbfb447,2021-03-30 01:10:00.000000 UTC,93.89.195.100,Bogatynia,Dolnośląskie,59,Firefox,Email,/cart,cart
1,1864178,NaN,3,a43078a4-1162-402b-8b5a-63c1ca9a6698,2020-01-25 05:29:00.000000 UTC,130.225.144.226,Bogatynia,Dolnośląskie,59,Chrome,Adwords,/cart,cart
2,563756,43423.0,3,d4f9811c-685c-4c76-bb7b-832988a74634,2023-04-13 17:07:24.125348 UTC,92.165.182.20,Bogatynia,Dolnośląskie,59,IE,Email,/cart,cart
3,527503,40683.0,4,707d9b5c-522a-49af-ba0e-b0c668ccd5ef,2023-03-26 03:27:24.000000 UTC,106.12.125.113,Bogatynia,Dolnośląskie,59,Safari,Email,/cart,cart
4,2261172,NaN,2,2df3b32c-4d95-4a8a-a448-af16eb9fc9a6,2021-08-05 05:45:00.000000 UTC,39.160.249.193,Bogatynia,Dolnośląskie,59,Other,Email,/cart,cart
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1240755,95505.0,4,dc58a296-4a6b-409f-837e-acb735e8d213,2021-09-06 13:28:14.000000 UTC,185.135.160.132,Wollongong,New South Wales,2530,Chrome,Email,/cart,cart
996,980805,75481.0,4,f838d70c-d4b9-4206-9cb4-27f7ab9a1420,2023-01-29 06:30:21.000000 UTC,73.47.179.232,Buckenbowra,New South Wales,2536,Firefox,Adwords,/cart,cart
997,569276,43833.0,6,4d6bbfa0-8d64-4606-a4ff-034270afcc29,2023-04-12 23:24:59.157286 UTC,12.80.137.205,Yambulla,New South Wales,2550,Safari,Email,/cart,cart
998,569269,43833.0,4,98b4d398-ea35-45dc-96ae-2e8dd3c8168b,2023-04-13 01:44:17.157286 UTC,119.4.192.107,Yambulla,New South Wales,2550,Chrome,Adwords,/cart,cart


Looking into web behavior the two main data points are browser,
and traffic source. The more important one to focus on would be
the actual traffic source that lead them to the site

In [36]:
q = "SELECT distinct(traffic_source) FROM EventsDF"
sqldf(q, globals())

,traffic_source
0,Email
1,Adwords
2,YouTube
3,Organic
4,Facebook


In [37]:
q = "SELECT user_id, traffic_source, count(*) as total FROM EventsDF WHERE user_id is not null GROUP BY user_id,traffic_source ORDER BY user_id DESC"
sqldf(q, globals())

,user_id,traffic_source,total
0,99996.0,Adwords,1
1,99996.0,Email,1
2,99986.0,Adwords,1
3,99986.0,Email,1
4,99986.0,Facebook,1
...,...,...,...
21348,22.0,Email,1
21349,20.0,Adwords,1
21350,17.0,Organic,1
21351,15.0,Email,1


What we can do with this view is segment the users by traffic source 
that lead them to the product, and plan on some tactics to keep
interest from the users high...for example those that frequently visiting based off email
can be sent target coupons and discounts via email to drive higher sales. Those
visiting based of social media like facebook and youtube ads, can be identified
and have trageted ads sent their way

In [38]:
q = "SELECT user_id, event_type, count(*) as total FROM EventsDF WHERE user_id is not null AND event_type in ('cart','purchase','cancel') GROUP BY user_id,event_type ORDER BY user_id DESC"
sqldf(q, globals())

,user_id,event_type,total
0,99986.0,cart,2
1,99981.0,cart,1
2,99957.0,cart,2
3,99936.0,cart,1
4,99934.0,cart,1
...,...,...,...
10465,22.0,cart,1
10466,20.0,cart,1
10467,17.0,cart,1
10468,15.0,cart,1


Using this view of how often a user puts items in a cart to 
actually purchasing, the company can dive deeper into which products 
specifically are most often put in the cart then not purchased.
These items can be given discount events, special advertising, or 
be removed from circulation if rarely purchased